# Next-Generation AI: Neural Cellular Automata Chatroom

This notebook demonstrates a futuristic AI concept: a neural cellular automata system that evolves in a dynamic environment, with real-time visualization and interactive controls. The principles here can inspire the architecture and features of a next-gen AI-powered chatroom.

## 1. Import Required Libraries
Import essential libraries such as numpy, matplotlib, and PyTorch for neural network operations and visualization.

In [ ]:
# Import essential libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import animation
from ipywidgets import interact, IntSlider, FloatSlider, Button, VBox, HBox
import copy

## 2. Define a Custom Neural Cellular Automata
Implement a neural network-based cellular automata class using PyTorch, where each cell's state is updated by a small neural network.

In [ ]:
class NeuralCellularAutomata(nn.Module):
    def __init__(self, state_dim=16, hidden_dim=64):
        super().__init__()
        self.state_dim = state_dim
        self.perceive = nn.Conv2d(state_dim, hidden_dim, kernel_size=3, padding=1)
        self.update = nn.Sequential(
            nn.Conv2d(hidden_dim, state_dim, kernel_size=1),
            nn.Tanh()
        )

    def forward(self, x):
        y = self.perceive(x)
        dx = self.update(y)
        return x + dx

# Example usage:
# nca = NeuralCellularAutomata()
# grid = torch.randn(1, 16, 32, 32)  # (batch, channels, height, width)
# new_grid = nca(grid)

## 3. Initialize a Dynamic Grid Environment
Create a 2D grid representing the environment, with random or user-defined initial states for each cell.

In [ ]:
def initialize_grid(batch_size=1, state_dim=16, height=32, width=32, seed=None):
    if seed is not None:
        np.random.seed(seed)
    grid = np.random.randn(batch_size, state_dim, height, width).astype(np.float32)
    return torch.tensor(grid)

# Example usage:
# grid = initialize_grid()

## 4. Evolve the Grid with AI-Driven Rules
Apply the neural cellular automata rules iteratively to evolve the grid, allowing the system to learn and adapt its update rules.

In [ ]:
def evolve_grid(nca, grid, steps=10):
    history = [grid.detach().cpu().numpy()]
    for _ in range(steps):
        grid = nca(grid)
        history.append(grid.detach().cpu().numpy())
    return grid, history

# Example usage:
# nca = NeuralCellularAutomata()
# grid = initialize_grid()
# final_grid, history = evolve_grid(nca, grid, steps=20)

## 5. Visualize the Evolution in Real-Time
Use matplotlib's animation capabilities to display the evolving grid in real-time within the notebook.

In [ ]:
def animate_evolution(history, channel=0):
    fig, ax = plt.subplots()
    ims = []
    for state in history:
        im = ax.imshow(state[0, channel], animated=True, cmap='viridis')
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=200, blit=True, repeat_delay=1000)
    plt.close(fig)
    return ani

# Example usage:
# from IPython.display import HTML
# ani = animate_evolution(history)
# HTML(ani.to_jshtml())

## 6. Interactive User Control for Evolution Parameters
Add interactive widgets (using ipywidgets) to let users adjust parameters such as learning rate, number of steps, and initial conditions during evolution.

In [ ]:
def interactive_evolution():
    nca = NeuralCellularAutomata()
    grid = initialize_grid()
    steps_slider = IntSlider(value=10, min=1, max=100, description='Steps')
    channel_slider = IntSlider(value=0, min=0, max=nca.state_dim-1, description='Channel')

    def update(steps, channel):
        final_grid, history = evolve_grid(nca, grid, steps)
        ani = animate_evolution(history, channel)
        from IPython.display import display, HTML
        display(HTML(ani.to_jshtml()))

    interact(update, steps=steps_slider, channel=channel_slider)

# Example usage:
# interactive_evolution()

## 7. Export Evolution Data for Further Analysis
Provide code to export the grid's evolution history as a dataset for further analysis or training.

In [ ]:
import pickle

def export_evolution_history(history, filename="evolution_history.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(history, f)
    print(f"Evolution history saved to {filename}")

# Example usage:
# export_evolution_history(history)